<a href="https://colab.research.google.com/github/khotimnr/SentiStrengthID/blob/master/Sentiment_Analysis_Khotim_(word_embedding_CBOW_bi_lstm).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install & Download


In [0]:
! pip install fasttext
! pip install -U gensim
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
# ! gunzip cc.id.300.bin.gz
! pip install -U symspellpy
! wget https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2
! pip install Sastrawi
# ! wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz
# ! gunzip cc.id.300.vec.gz
! pip install wordcloud
# ! pip install bpemb

     |████████████████████████████████| 61kB 3.6MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.1-cp36-cp36m-linux_x86_64.whl size=2384395 sha256=b311ce0d1fad6538a6def3b4abf82fbce2879d8c2f5533144999f0cc0aa13c0e
  Stored in directory: /root/.cache/pip/wheels/9f/f0/04/caa82c912aee89ce76358ff954f3f0729b7577c8ff23a292e3
Successfully built fasttext
     |████████████████████████████████| 24.2MB 42.0MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0
     |████████████████████████████████| 2.6MB 6.5MB/s 
--2019-11-12 09:37:58--  https://dumps.wikimedia.org/idwiki/latest/idwiki-latest-pages-articles.xml.bz2
Resolving dumps.wikimedia.org (dumps.wikimedia.org)... 208.80.154.7, 2620:0:861:1:208:80:154:7
Connecting to dumps.wikimedia.org (dumps.wikimedia.org)|208.80.154.7|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548827007 (523M) [application/octet-stream]
Saving to: ‘idwiki-latest-p

# Import Library

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

TensorFlow 2.x selected.


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from gensim.corpora import WikiCorpus
from urllib.request import urlopen
from collections import Counter
import gzip
import numpy as np

# Load Dataset

In [0]:
# lakukan load dataset terlebih dahulu
df_review = pd.read_csv("review.csv")
df_review.head()

FileNotFoundError: ignored

In [0]:
# coba kita perhatikan berapa banyak data yang kita dapatkan
df_review.describe()

In [0]:
# coba kita lihat berapa banyak jumlah data untuk tiap rate
df_review["Star"].value_counts()

In [0]:
df_review['Comment'][1000]

In [0]:
# drop unused column
df_preprocessed = df_review.copy()
df_preprocessed = df_preprocessed.drop(columns=['Date', 'Name'])
df_preprocessed.head()

# Preprocessing

1. Label Encoding (Sentiment positive and negative)
2. Balancing Dataset
3. Spell check (Wolf Garbe - Symspell)
4. Stemming/Lemmatization (Sastrawi)
5. Word embedding (fasttext pre-trained model bahasa indonesia, CBOW)

## Label Encoding

In [0]:
# label encoding pada Star
# Jika 1-4, maka sentiment negative (label 0)
# Jika 5, maka sentiment positive (label 1)
label = []
for index, row in df_preprocessed.iterrows():
    if row["Star"] == 5:
        label.append(1)
    else:
        label.append(0)

df_preprocessed["label"] = label
df_preprocessed = df_preprocessed.drop(columns=['Star'])
df_preprocessed.head()

In [0]:
df_preprocessed["label"].value_counts()

## Balancing Dataset

In [0]:
# balancing dataset
s_1 = df_preprocessed[df_preprocessed['label']==0].sample(25000,replace=True)
s_2 = df_preprocessed[df_preprocessed['label']==1].sample(25000,replace=True)
df_preprocessed = pd.concat([s_1, s_2])

print(df_preprocessed.shape)

print(df_preprocessed['label'].value_counts(normalize=True))

In [0]:
Counter(df_preprocessed["label"])

In [0]:
# wordcloud
from wordcloud import WordCloud

# Polarity == 0 negative
train_s0 = df_preprocessed[df_preprocessed["label"] == 0]
all_text_s0 = ' '.join(word for word in train_s0["Comment"])
wordcloud = WordCloud(colormap='Reds', width=1000, height=1000, mode='RGBA', background_color='white').generate(all_text_s0)
plt.figure(figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

# Polarity == 1 positive
train_s1 = df_preprocessed[df_preprocessed["label"] == 1]
all_text_s1 = ' '.join(word for word in train_s1["Comment"])
wordcloud = WordCloud(width=1000, height=1000, colormap='Blues', background_color='white', mode='RGBA').generate(all_text_s1)
plt.figure( figsize=(20,10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.margins(x=0, y=0)
plt.show()

## Spell check

In [0]:
# create txt file for spell check dictionary
wiki = WikiCorpus("idwiki-latest-pages-articles.xml.bz2", lemmatize=False, dictionary={})

with open("wiki-id-formatted.txt", 'w', encoding="utf8") as output:
     counter = 0
     for text in wiki.get_texts():
         output.write(' '.join(text)+"\n")
         counter = counter + 1
         if counter > 200000: # Hanya mengambil 200,000 artikel pertama dari total artikel sekitar 300,000
             break

NameError: ignored

In [0]:
# create dictionary for symspell
from symspellpy import SymSpell

sym_spell = SymSpell()
corpus_path = "wiki-id-formatted.txt"
sym_spell.create_dictionary(corpus_path)

In [0]:
from symspellpy import SymSpell, Verbosity

input_term = "diperbaiko diperbaike perbaiknn"  # misspelling of "apostrophe"
# max edit distance per lookup
# (max_edit_distance_lookup <= max_dictionary_edit_distance)
suggestions = sym_spell.lookup_compound(input_term, max_edit_distance=2)

for suggestion in suggestions:
    print(suggestion.term)

In [0]:
review = []
for index, row in df_preprocessed.iterrows():
    suggestions = sym_spell.lookup_compound(row["Comment"], max_edit_distance=2)
    review.append(suggestions[0].term)
    
df_preprocessed["Comment"] = review
df_preprocessed.head()

## Stemming/Lemmatization

In [0]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# contoh
kalimat = 'lemot aplikasinya sedih high banget aku fiturnya harus diperbaiki supaya lebih mudah lagi tampilan kurang menarik'
katadasar = stemmer.stem(kalimat)
 
print(katadasar)

In [0]:
review = []
for index, row in df_preprocessed.iterrows():
    review.append(stemmer.stem(row["Comment"]))
    
df_preprocessed["Comment"] = review
df_preprocessed.head()

In [0]:
def preprocess_data(text):
    # spell check
    text = sym_spell.lookup_compound(text, max_edit_distance=2)[0].term

    # stemming
    text = stemmer.stem(text)

    return text

## Word Embedding

In [0]:
file = gzip.open(urlopen('https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.vec.gz'))

vocab_and_vectors = {}
# put words as dict indexes and vectors as words values
for line in file:
    values = line.split()
    word = values [0].decode('utf-8')
    vector = np.asarray(values[1:], dtype='float32')
    vocab_and_vectors[word] = vector

In [0]:
# more imports
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# how many features should the tokenizer extract
features = 500
tokenizer = Tokenizer(num_words = features)
# fit the tokenizer on our text
tokenizer.fit_on_texts(df_preprocessed["Comment"].tolist())

# get all words that the tokenizer knows
word_index = tokenizer.word_index
print(len(word_index))
# put the tokens in a matrix
X = tokenizer.texts_to_sequences(df_preprocessed["Comment"].tolist())
X = pad_sequences(X)

# prepare the labels
y = df_preprocessed["label"].values

# split in train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=30)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in word_index.items():
    embedding_vector = vocab_and_vectors.get(word)
    # words that cannot be found will be set to 0
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [0]:
embedding_matrix.shape

# Training

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(len(word_index)+1, 300, input_length=X.shape[1], weights=[embedding_matrix], trainable=False),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.summary()
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [0]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_test, y_test), 
                    validation_steps=30)

# Evaluation

In [0]:
test_loss, test_acc = model.evaluate(X_test, y_test)

print('Test Loss: {}'.format(test_loss))
print('Test Accuracy: {}'.format(test_acc))

In [0]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [0]:
plot_graphs(history, 'accuracy')

In [0]:
plot_graphs(history, 'loss')

In [0]:
# predict on a sample text
# jika >= 0.5, sentiment positif
# jika < 0.5, sentiment negatif
sample_positive_review = "Respon Cepat, pelayanan bagus"
sample_negative_review = "pendaftaran buka reksanya lama sekali sudah hampir dua minggu belum selesai juga"

X_pos = tokenizer.texts_to_sequences([preprocess_data(sample_positive_review)])
X_pos = pad_sequences(X_pos, maxlen=X.shape[1])

X_neg = tokenizer.texts_to_sequences([preprocess_data(sample_negative_review)])
X_neg = pad_sequences(X_neg, maxlen=X.shape[1])

pred_pos = model.predict(X_pos)
pred_neg = model.predict(X_neg)

print(pred_pos)
print(pred_neg)

## Confusion matrix

In [0]:
# import library evaluation
from sklearn.metrics import f1_score, recall_score, precision_score, confusion_matrix, accuracy_score

In [0]:
# lakukan prediksi pada data test
pred = model.predict(X_test)

# rubah sigmoid
y_pred = []
for p in pred:
    if p >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [0]:
# f1_score
f1_score(y_test, y_pred)

In [0]:
# accuracy score
accuracy_score(y_test, y_pred)

In [0]:
# precision score
precision_score(y_test, y_pred)

In [0]:
# recall score
recall_score(y_test, y_pred)

In [0]:
# confusion matrix
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
tn, fp, fn, tp